In [1]:
!git clone https://github.com/google-research/bert.git


Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 340 (delta 203), reused 303 (delta 185), pack-reused 0
Receiving objects: 100% (340/340), 192.58 KiB | 5.35 MiB/s, done.
Resolving deltas: 100% (203/203), done.


In [2]:
%cd bert


/content/bert


In [5]:
from datasets import load_dataset
import os

# Load the MRPC dataset
dataset = load_dataset("glue", "mrpc")

# Create the 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Convert datasets to pandas DataFrames
train_df = dataset['train'].to_pandas()
val_df = dataset['validation'].to_pandas()

# Save the data as TSV files
train_df[['sentence1', 'sentence2', 'label']].to_csv('data/train.tsv', index=False, header=False, sep='\t')
val_df[['sentence1', 'sentence2', 'label']].to_csv('data/dev.tsv', index=False, header=False, sep='\t')

print("Data saved successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Data saved successfully.


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("glue", "mrpc")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['validation']      # evaluation dataset
)


In [9]:
trainer.train()


Step,Training Loss
10,0.780200
20,0.787400
30,0.762200
40,0.746700
50,0.707300
60,0.666500
70,0.637600
80,0.629300
90,0.581200
100,0.683100


TrainOutput(global_step=1377, training_loss=0.39418783539347485, metrics={'train_runtime': 1018.4023, 'train_samples_per_second': 10.805, 'train_steps_per_second': 1.352, 'total_flos': 2895274053181440.0, 'train_loss': 0.39418783539347485, 'epoch': 3.0})

In [10]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.6390026211738586, 'eval_runtime': 11.1132, 'eval_samples_per_second': 36.713, 'eval_steps_per_second': 4.589, 'epoch': 3.0}


In [6]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import os

# Load the MRPC dataset
dataset = load_dataset("glue", "mrpc")

# Tokenize the dataset
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Create the 'results' directory if it doesn't exist
os.makedirs('results', exist_ok=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args_lr = TrainingArguments(
    output_dir='./results_lr',          # output directory
    num_train_epochs=3,                 # number of training epochs
    per_device_train_batch_size=16,     # batch size for training
    per_device_eval_batch_size=16,      # batch size for evaluation
    learning_rate=5e-5,                 # increased learning rate
    warmup_steps=500,                   # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                  # strength of weight decay
    logging_dir='./logs_lr',            # directory for storing logs
    logging_steps=10,
)

trainer_lr = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args_lr,               # training arguments, defined above
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['validation']      # evaluation dataset
)

# Train with the increased learning rate
trainer_lr.train()
eval_results_lr = trainer_lr.evaluate()
print("Learning Rate Adjustment Results:", eval_results_lr)


Step,Training Loss
10,0.736900
20,0.746700
30,0.713200
40,0.695200
50,0.696100
60,0.669500
70,0.626900
80,0.646800
90,0.610000
100,0.594100


Learning Rate Adjustment Results: {'eval_loss': 0.5363577008247375, 'eval_runtime': 11.7999, 'eval_samples_per_second': 34.577, 'eval_steps_per_second': 2.203, 'epoch': 3.0}


In [8]:
training_args_bs = TrainingArguments(
    output_dir='./results_bs',           # output directory
    num_train_epochs=3,                  # number of training epochs
    per_device_train_batch_size=8,       # decreased batch size for training
    per_device_eval_batch_size=8,        # decreased batch size for evaluation
    learning_rate=2e-5,                  # default learning rate
    warmup_steps=500,                    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # strength of weight decay
    logging_dir='./logs_bs',             # directory for storing logs
    logging_steps=10,
)

trainer_bs = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args_bs,               # training arguments, defined above
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['validation']      # evaluation dataset
)

# Train with the decreased batch size
trainer_bs.train()
eval_results_bs = trainer_bs.evaluate()
print("Batch Size Adjustment Results:", eval_results_bs)


Step,Training Loss
10,0.179400
20,0.050200
30,0.038300
40,0.102300
50,0.200900
60,0.079500
70,0.111100
80,0.062100
90,0.096800
100,0.198000


Batch Size Adjustment Results: {'eval_loss': 0.959290623664856, 'eval_runtime': 11.5022, 'eval_samples_per_second': 35.472, 'eval_steps_per_second': 4.434, 'epoch': 3.0}


In [9]:
training_args_epochs = TrainingArguments(
    output_dir='./results_epochs',       # output directory
    num_train_epochs=5,                  # increased number of training epochs
    per_device_train_batch_size=16,      # batch size for training
    per_device_eval_batch_size=16,       # batch size for evaluation
    learning_rate=2e-5,                  # default learning rate
    warmup_steps=500,                    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # strength of weight decay
    logging_dir='./logs_epochs',         # directory for storing logs
    logging_steps=10,
)

trainer_epochs = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args_epochs,           # training arguments, defined above
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['validation']      # evaluation dataset
)

# Train with the increased number of epochs
trainer_epochs.train()
eval_results_epochs = trainer_epochs.evaluate()
print("Number of Epochs Adjustment Results:", eval_results_epochs)


Step,Training Loss
10,0.000600
20,0.002800
30,0.046000
40,0.061500
50,0.018700
60,0.001800
70,0.101500
80,0.009000
90,0.009100
100,0.000600


Number of Epochs Adjustment Results: {'eval_loss': 1.069010853767395, 'eval_runtime': 11.8558, 'eval_samples_per_second': 34.414, 'eval_steps_per_second': 2.193, 'epoch': 5.0}


In [10]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load BERT-Large model and tokenizer
model_name = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir='./results_large',        # output directory
    num_train_epochs=3,                  # number of training epochs
    per_device_train_batch_size=8,       # batch size for training (adjusted for larger model)
    per_device_eval_batch_size=8,        # batch size for evaluation
    learning_rate=2e-5,                  # learning rate
    warmup_steps=500,                    # number of warmup steps
    weight_decay=0.01,                   # strength of weight decay
    logging_dir='./logs_large',          # directory for storing logs
    logging_steps=10,
)


In [12]:
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['validation']      # evaluation dataset
)

trainer.train()
eval_results = trainer.evaluate()
print("Evaluation Results for BERT-Large:", eval_results)


Step,Training Loss
10,0.872400
20,0.844600
30,0.808100
40,0.849000
50,0.867500
60,0.755300
70,0.751400
80,0.742800
90,0.707400
100,0.706800


Evaluation Results for BERT-Large: {'eval_loss': 0.5476898550987244, 'eval_runtime': 38.049, 'eval_samples_per_second': 10.723, 'eval_steps_per_second': 1.34, 'epoch': 3.0}
